In [20]:
class Feistel:
    def __init__(self, data, key = "f2000000", cipher = "0"):
        con = Converter()
        self.data = con.hexToBin64(data)
        self.cipher = con.hexToBin64(cipher)
        self.key = con.hexToBin32(key)

        
    def oneRound(self, rounds):
        con = Converter()

        leftText = self.data[0:32]
        rightText = self.data[32:64]
        
        leftKey = self.key[0:16]
        rightKey = self.key[16:32]

        #print("original key : {0}".format(con.binToHex(self.key)))
        #print("original plaintext : {0}".format(con.binToHex(self.data)))
        #print("original left : {0}".format(con.binToHex(leftText)))
        #print("original right : {0}".format(con.binToHex(rightText)))
        
        for j in range(rounds):

            xorInter1 = ""
            xorInter2 = ""
            xorInter3 = ""

            #subkey progress
            subKey = leftKey[j+1:16] + leftKey[0:j+1] + rightKey[j+1:16] + rightKey[0:j+1]
            for i in range(32):
                xorInter1 += str(int(rightText[i])^int(subKey[i]))
                
            #Substitution / choice(S-box)

            sBox = [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
                   [5, 6, 12, 15, 8, 10, 0, 4, 11, 3, 7, 13, 14, 1, 2, 9]]

            #S-box progress
            for i in range(8):
                selectBit = xorInter1[i*4:(i+1)*4]

                #binary into int
                convInt = int(selectBit, 2)
                findKey = sBox[i%2][convInt]
                
                #int into binary
                xorInter2 += bin(findKey)[2:].zfill(4)#"{0:b}".format(findKey)
                
            #permutation    
            permutation = (
                xorInter2[21]+xorInter2[1]+xorInter2[14]+xorInter2[24]+xorInter2[9]+xorInter2[26]+xorInter2[7]+xorInter2[28]+
                xorInter2[3]+xorInter2[18]+xorInter2[17]+xorInter2[27]+xorInter2[10]+xorInter2[20]+xorInter2[29]+xorInter2[23]+
                xorInter2[31]+xorInter2[2]+xorInter2[8]+xorInter2[11]+xorInter2[15]+xorInter2[12]+xorInter2[5]+xorInter2[22]+
                xorInter2[30]+xorInter2[19]+xorInter2[13]+xorInter2[25]+xorInter2[6]+xorInter2[0]+xorInter2[4]+xorInter2[16])
            
            for i in range(32):
                xorInter3 += str(int(permutation[i])^int(leftText[i]))

            
            #final shift left 32bit <-> right 32bit
            leftText = rightText
            rightText = xorInter3
            #print("{0} round Left: {1}".format(j, con.binToHex(leftText)))
            #print("{0} round Right: {1}".format(j, con.binToHex(rightText)))
        
        temp = rightText
        rightText = leftText
        leftText = temp
        temp = leftText + rightText
        
        #print("ciphertext: {0}".format(con.binToHex16(temp)))
        return temp
        
    def findKey(self, rounds=10):
        con = Converter()
        one = Feistel(con.binToHex16(self.data), con.binToHex(self.key), con.binToHex16(self.cipher))
        if self.cipher != "0":
            answer = one.oneRound(rounds)
            while self.cipher != answer:
                self.key = bin(int(self.key, 2) + 1)[2:].zfill(32)
                one = Feistel(con.binToHex16(self.data), con.binToHex(self.key), con.binToHex16(self.cipher))
                answer = one.oneRound(rounds)
                
        return con.binToHex(self.key)
                
            
            
        

    
class Converter:

    def hexToBin64(self, data):
        
        return bin(int(data, 16))[2:].zfill(64)
    
    def hexToBin32(self, data):
        
        return bin(int(data, 16))[2:].zfill(32)
    
    def binToHex(self, data):

        #return '%08X' %int(data, 2)
        return hex(int(data, 2))[2:].zfill(8)
    
    def binToHex16(self, data):
        
        return hex(int(data, 2))[2:].zfill(16)

    def binToHexBySix(self, data):
        bySix = ""
        for i in range(8):
            bySix += '%02X' %int(data[i*6:i*6+6], 2)
            
        return bySix


In [21]:
feis1 = Feistel("2174c501ac12f9d1","f2000000" ,"df9fcc3ffe09809d")

print(feis1.findKey())

feis2 = Feistel("40ff24330947f610","f2000000" ,"ec2de1305b5f5b02")

print(feis2.findKey())


f24cb904
f24cb904


In [17]:
feis = Feistel("770245ffbad4173e", "12345678")

feis.oneRound(10)

original key : 12345678
original plaintext : 770245ffbad4173e
original left : 770245ff
original right : bad4173e
0 round Left: bad4173e
0 round Right: df372142
1 round Left: df372142
1 round Right: bfd3d48b
2 round Left: bfd3d48b
2 round Right: 3bf78daf
3 round Left: 3bf78daf
3 round Right: 65c07511
4 round Left: 65c07511
4 round Right: f85c0a90
5 round Left: f85c0a90
5 round Right: 489c499e
6 round Left: 489c499e
6 round Right: 89d06d07
7 round Left: 89d06d07
7 round Right: 16aa5c2a
8 round Left: 16aa5c2a
8 round Right: 2dac7c0b
9 round Left: 2dac7c0b
9 round Right: 4179183b
ciphertext: 4179183b2dac7c0b


'0100000101111001000110000011101100101101101011000111110000001011'

In [18]:
feis = Feistel("1024aa9f473c58c1", "0434ef71")

feis.oneRound(10)

original key : 0434ef71
original plaintext : 1024aa9f473c58c1
original left : 1024aa9f
original right : 473c58c1
0 round Left: 473c58c1
0 round Right: 0b7edecd
1 round Left: 0b7edecd
1 round Right: b973db12
2 round Left: b973db12
2 round Right: bd1f22e3
3 round Left: bd1f22e3
3 round Right: f5ef264b
4 round Left: f5ef264b
4 round Right: af321dad
5 round Left: af321dad
5 round Right: 05df7929
6 round Left: 05df7929
6 round Right: 7d02246f
7 round Left: 7d02246f
7 round Right: 0ba19860
8 round Left: 0ba19860
8 round Right: c78f6ef0
9 round Left: c78f6ef0
9 round Right: 4fc83760
ciphertext: 4fc83760c78f6ef0


'0100111111001000001101110110000011000111100011110110111011110000'

original key : 98765432
original plaintext : 770245ffbad4173a
original left : 770245ff
original right : bad4173a
0 round Left: bad4173a
0 round Right: 41b71f66
1 round Left: 41b71f66
1 round Right: fd19ad2e
2 round Left: fd19ad2e
2 round Right: d85916ff
3 round Left: d85916ff
3 round Right: afef74c9
4 round Left: afef74c9
4 round Right: 3d2fc017
5 round Left: 3d2fc017
5 round Right: 48516f8d
6 round Left: 48516f8d
6 round Right: 332dca08
7 round Left: 332dca08
7 round Right: 75260a57
8 round Left: 75260a57
8 round Right: 3c7d0b57
9 round Left: 3c7d0b57
9 round Right: c9000e27
ciphertext: c9000e273c7d0b57


'1100100100000000000011100010011100111100011111010000101101010111'